## VIBE Image Editing Demo

This notebook demonstrates how to use the VIBE framework for text-guided image editing.

### Prerequisites

Make sure you have installed the dependencies as described in the `README.md`.
You also need the pipeline checkpoint.

In [ ]:
# Uncomment and run if you haven't installed dependencies yet
# !pip install -r requirements/requirements-dev.txt

In [ ]:
import os
import sys

import torch
from matplotlib import pyplot as plt
from PIL import Image

# Add the current directory to the path to import vibe modules
sys.path.append(os.path.abspath("."))

from vibe.editor import ImageEditor

### Configuration

Set the paths for the input image, checkpoint, and output directory.  
**Important:** You must set `CHECKPOINT_PATH` to the location of your local pipeline checkpoint.

In [ ]:
# Configuration
IMAGE_PATH = "examples/dog.jpg"
EDITING_PROMPT = "Make the dog look like a painting"
CHECKPOINT_PATH = "/path/to/pipeline/checkpoint"  # <--- UPDATE THIS PATH
OUTPUT_PATH = "outputs/"

IMAGE_GUIDANCE_SCALE = 1.2  # input image condition guidance
GUIDANCE_SCALE = 4.5  # prompt condition guidance

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

## Load Input Image

In [ ]:
if not os.path.exists(IMAGE_PATH):
    print(f"Image not found at {IMAGE_PATH}")
    # You can upload an image or change the path
else:
    original_image = Image.open(IMAGE_PATH).convert("RGB")
    plt.figure()
    plt.imshow(original_image)
    plt.axis("off")
    plt.title("Original Image")
    plt.show()

## Initialize Image Editor

This will load the model weights. It might take a moment.

In [ ]:
try:
    editor = ImageEditor(
        checkpoint_path=CHECKPOINT_PATH,
        image_guidance_scale=IMAGE_GUIDANCE_SCALE,
        guidance_scale=GUIDANCE_SCALE,
        num_inference_steps=20,
        device=DEVICE,
    )
    print("Editor initialized successfully!")
except Exception as exp:
    print(f"Error initializing editor: {exp}")
    print("Please check if CHECKPOINT_PATH is correct and points to a valid model checkpoint.")

## Run Inference

In [ ]:
if "editor" in locals() and "original_image" in locals():
    print(f"Editing with prompt: '{EDITING_PROMPT}'...")

    try:
        # Generate edited images
        edited_images = editor.generate_edited_image(original_image, EDITING_PROMPT, num_images_per_prompt=1)

    except Exception as e:
        print(f"Error during generation: {e}")
else:
    print("Editor or image not loaded correctly.")

In [ ]:
# Display results
plt.figure(figsize=(14, 7))

# Original
plt.subplot(1, 2, 1)
plt.imshow(original_image)
plt.axis("off")
plt.title("Original")

# Edited
plt.subplot(1, 2, 2)
plt.imshow(edited_images[0])
plt.axis("off")
plt.title(f"Edited: {EDITING_PROMPT}")

plt.tight_layout()
plt.show()

In [ ]:
# Save result
os.makedirs(OUTPUT_PATH, exist_ok=True)
save_path = os.path.join(OUTPUT_PATH, "edited_image.jpg")
edited_images[0].save(save_path)
print(f"Saved result to {save_path}")